<a href="https://colab.research.google.com/github/Jurgo001/TH_TriTueNhanTao/blob/main/Buoi03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
import copy
import math
import numpy as np # Dùng để in bàn cờ đẹp hơn

# --- I. CÁC HẰNG SỐ VÀ KHỞI TẠO ---
KY_HIEU_X = "X"
KY_HIEU_O = "O"
O_TRONG = None

def trang_thai_ban_dau():
    """Trả về trạng thái ban đầu của bàn cờ 3x3 rỗng."""
    return [[O_TRONG, O_TRONG, O_TRONG],
            [O_TRONG, O_TRONG, O_TRONG],
            [O_TRONG, O_TRONG, O_TRONG]]

# --- II. CÁC HÀM CƠ BẢN CỦA TRÒ CHƠI ---

def nguoi_choi(ban_co):
    """Xác định người chơi tiếp theo. X luôn đi trước."""
    dem_x = sum(hang.count(KY_HIEU_X) for hang in ban_co)
    dem_o = sum(hang.count(KY_HIEU_O) for hang in ban_co)

    # Nếu số lượng X bằng O, đến lượt X. Ngược lại, đến lượt O.
    return KY_HIEU_X if dem_x == dem_o else KY_HIEU_O

def cac_hanh_dong(ban_co):
    """Trả về tập hợp tất cả các nước đi khả thi (i, j) trên bàn cờ."""
    hanh_dong_kha_thi = set()
    for i in range(3):
        for j in range(3):
            if ban_co[i][j] == O_TRONG:
                hanh_dong_kha_thi.add((i, j))
    return hanh_dong_kha_thi

def ket_qua_nuoc_di(ban_co, hanh_dong):
    """Trả về trạng thái bàn cờ mới sau khi thực hiện nước đi (i, j)."""
    if hanh_dong not in cac_hanh_dong(ban_co):
        raise Exception("Nước đi không hợp lệ")

    nguoi_choi_hien_tai = nguoi_choi(ban_co)
    ban_co_moi = copy.deepcopy(ban_co)
    (i, j) = hanh_dong
    ban_co_moi[i][j] = nguoi_choi_hien_tai
    return ban_co_moi

def tim_nguoi_thang(ban_co):
    """Kiểm tra và trả về người chiến thắng (X hoặc O), nếu có."""

    # Kiểm tra 3 hàng, 3 cột
    for i in range(3):
        # Kiểm tra Hàng (row): ban_co[i][0] == ban_co[i][1] == ban_co[i][2]
        if ban_co[i][0] == ban_co[i][1] == ban_co[i][2] and ban_co[i][0] is not O_TRONG:
            return ban_co[i][0]
        # Kiểm tra Cột (column): ban_co[0][i] == ban_co[1][i] == ban_co[2][i]
        if ban_co[0][i] == ban_co[1][i] == ban_co[2][i] and ban_co[0][i] is not O_TRONG:
            return ban_co[0][i]

    # Kiểm tra 2 đường chéo
    # Chéo chính (\): [0][0], [1][1], [2][2]
    if ban_co[0][0] == ban_co[1][1] == ban_co[2][2] and ban_co[0][0] is not O_TRONG:
        return ban_co[0][0]
    # Chéo phụ (/): [0][2], [1][1], [2][0]
    if ban_co[0][2] == ban_co[1][1] == ban_co[2][0] and ban_co[0][2] is not O_TRONG:
        return ban_co[0][2]

    return None

def ket_thuc(ban_co):
    """Trả về True nếu trò chơi kết thúc (có người thắng hoặc hòa), False nếu ngược lại."""
    if tim_nguoi_thang(ban_co) is not None:
        return True
    # Kiểm tra nếu bàn cờ đầy (Hòa)
    return not any(O_TRONG in hang for hang in ban_co)

def diem_so(ban_co):
    """Trả về điểm số: 1 nếu X thắng, -1 nếu O thắng, 0 nếu hòa."""
    nguoi_thang = tim_nguoi_thang(ban_co)
    if nguoi_thang == KY_HIEU_X:
        return 1
    elif nguoi_thang == KY_HIEU_O:
        return -1
    return 0

# --- III. THUẬT TOÁN MINIMAX ---

def gia_tri_toi_da(trang_thai):
    """Tính giá trị tối đa (cho người chơi X) có thể đạt được từ trạng thái hiện tại."""
    if ket_thuc(trang_thai):
        return diem_so(trang_thai)
    v = -math.inf
    for hanh_dong in cac_hanh_dong(trang_thai):
        # X cố gắng tối đa hóa (max) kết quả từ O (minValue)
        v = max(v, gia_tri_toi_thieu(ket_qua_nuoc_di(trang_thai, hanh_dong)))
    return v

def gia_tri_toi_thieu(trang_thai):
    """Tính giá trị tối thiểu (cho người chơi O) có thể đạt được từ trạng thái hiện tại."""
    if ket_thuc(trang_thai):
        return diem_so(trang_thai)
    v = math.inf
    for hanh_dong in cac_hanh_dong(trang_thai):
        # O cố gắng tối thiểu hóa (min) kết quả từ X (maxValue)
        v = min(v, gia_tri_toi_da(ket_qua_nuoc_di(trang_thai, hanh_dong)))
    return v

def minimax(ban_co):
    """Trả về nước đi tối ưu (optimal action) cho người chơi hiện tại."""
    nguoi_choi_hien_tai = nguoi_choi(ban_co)

    if nguoi_choi_hien_tai == KY_HIEU_X:
        # X là người chơi MAX
        diem_tot_nhat = -math.inf
        hanh_dong_tot_nhat = None
        for hanh_dong in cac_hanh_dong(ban_co):
            gia_tri = gia_tri_toi_thieu(ket_qua_nuoc_di(ban_co, hanh_dong))
            if gia_tri > diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
        return hanh_dong_tot_nhat

    else: # người chơi hiện tại là O
        # O là người chơi MIN
        diem_tot_nhat = math.inf
        hanh_dong_tot_nhat = None
        for hanh_dong in cac_hanh_dong(ban_co):
            gia_tri = gia_tri_toi_da(ket_qua_nuoc_di(ban_co, hanh_dong))
            if gia_tri < diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
        return hanh_dong_tot_nhat

# --- IV. KHỐI THỰC THI TRÒ CHƠI (MAIN) ---

if __name__ == "__main__":
    ban_co = trang_thai_ban_dau()
    print("--- TRÒ CHƠI TIC TAC TOE (AI) ---")

    # Chọn người chơi
    while True:
        lua_chon = input("Bạn muốn chơi với vai trò X hay O? (X/O): ").upper()
        if lua_chon in [KY_HIEU_X, KY_HIEU_O]:
            nguoi_dung = lua_chon
            ai = KY_HIEU_O if nguoi_dung == KY_HIEU_X else KY_HIEU_X
            break
        print("Lựa chọn không hợp lệ. Vui lòng nhập X hoặc O.")

    # Vòng lặp trò chơi
    while True:
        ket_thuc_game = ket_thuc(ban_co)
        luot_hien_tai = nguoi_choi(ban_co)

        # In bàn cờ hiện tại
        print("\n--- Bàn cờ hiện tại ---")
        print(np.array(ban_co))

        if ket_thuc_game:
            nguoi_thang = tim_nguoi_thang(ban_co)
            if nguoi_thang is None:
                print("Trò chơi kết thúc: HÒA.")
            else:
                print(f"Trò chơi kết thúc: {nguoi_thang} THẮNG.")
            break

        # Lượt của AI
        if luot_hien_tai == ai:
            print(f"Máy tính ({ai}) đang tính toán nước đi tối ưu...")
            nuoc_di_ai = minimax(ban_co)
            ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_ai)
            print(f"Máy tính đi: Hàng {nuoc_di_ai[0]}, Cột {nuoc_di_ai[1]}")

        # Lượt của Người chơi
        else:
            print(f"Lượt của bạn ({nguoi_dung})")
            while True:
                try:
                    i = int(input("Nhập Hàng (0-2): "))
                    j = int(input("Nhập Cột (0-2): "))
                    nuoc_di_nguoi_dung = (i, j)

                    if nuoc_di_nguoi_dung in cac_hanh_dong(ban_co):
                        ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_nguoi_dung)
                        break
                    else:
                        print("Nước đi không hợp lệ hoặc ô đã bị chiếm. Vui lòng thử lại.")
                except ValueError:
                    print("Vui lòng chỉ nhập số (0, 1, hoặc 2).")2

--- TRÒ CHƠI TIC TAC TOE (AI) ---
Bạn muốn chơi với vai trò X hay O? (X/O): x

--- Bàn cờ hiện tại ---
[[None None None]
 [None None None]
 [None None None]]
Lượt của bạn (X)
Nhập Hàng (0-2): 1
Nhập Cột (0-2): 1

--- Bàn cờ hiện tại ---
[[None None None]
 [None 'X' None]
 [None None None]]
Máy tính (O) đang tính toán nước đi tối ưu...
Máy tính đi: Hàng 0, Cột 0

--- Bàn cờ hiện tại ---
[['O' None None]
 [None 'X' None]
 [None None None]]
Lượt của bạn (X)
Nhập Hàng (0-2): 1
Nhập Cột (0-2): 
Vui lòng chỉ nhập số (0, 1, hoặc 2).
Nhập Hàng (0-2): 2
Nhập Cột (0-2): 1

--- Bàn cờ hiện tại ---
[['O' None None]
 [None 'X' None]
 [None 'X' None]]
Máy tính (O) đang tính toán nước đi tối ưu...
Máy tính đi: Hàng 0, Cột 1

--- Bàn cờ hiện tại ---
[['O' 'O' None]
 [None 'X' None]
 [None 'X' None]]
Lượt của bạn (X)
Nhập Hàng (0-2): 1
Nhập Cột (0-2): 2

--- Bàn cờ hiện tại ---
[['O' 'O' None]
 [None 'X' 'X']
 [None 'X' None]]
Máy tính (O) đang tính toán nước đi tối ưu...
Máy tính đi: Hàng 1, Cột 0

--

In [8]:
# -*- coding: utf-8 -*-
import copy
import math
import numpy as np

# --- I. CÁC HẰNG SỐ VÀ KHỞI TẠO ---
KY_HIEU_X = "X"
KY_HIEU_O = "O"
O_TRONG = None
KICH_THUOC = 4 # Kích thước bàn cờ 4x4

def trang_thai_ban_dau():
    """Trả về trạng thái ban đầu của bàn cờ 4x4 rỗng."""
    return [[O_TRONG for _ in range(KICH_THUOC)] for _ in range(KICH_THUOC)]

# --- II. CÁC HÀM CƠ BẢN CỦA TRÒ CHƠI ---

def nguoi_choi(ban_co):
    """Xác định người chơi tiếp theo theo quy tắc chuẩn: X đi trước."""
    dem_x = sum(hang.count(KY_HIEU_X) for hang in ban_co)
    dem_o = sum(hang.count(KY_HIEU_O) for hang in ban_co)

    return KY_HIEU_X if dem_x == dem_o else KY_HIEU_O

def cac_hanh_dong(ban_co):
    """Trả về tập hợp tất cả các nước đi khả thi (hàng, cột) trong ô trống."""
    hanh_dong_kha_thi = set()
    for i in range(KICH_THUOC):
        for j in range(KICH_THUOC):
            if ban_co[i][j] == O_TRONG:
                hanh_dong_kha_thi.add((i, j))
    return hanh_dong_kha_thi

def ket_qua_nuoc_di(ban_co, hanh_dong):
    """Trả về trạng thái bàn cờ mới sau khi người chơi hiện tại thực hiện nước đi."""
    nguoi_choi_hien_tai = nguoi_choi(ban_co)
    ban_co_moi = copy.deepcopy(ban_co)
    (i, j) = hanh_dong
    ban_co_moi[i][j] = nguoi_choi_hien_tai
    return ban_co_moi

# --- III. HÀM KIỂM TRA THẮNG THUA ---
# (Giữ nguyên logic kiểm tra thắng cho 4x4)

def tim_nguoi_thang(ban_co):
    """Kiểm tra và trả về người chiến thắng nếu có 4 ô liên tiếp."""
    N = KICH_THUOC

    # 1. Kiểm tra Hàng (Ngang) và Cột (Dọc)
    for i in range(N):
        if ban_co[i][0] == ban_co[i][1] == ban_co[i][2] == ban_co[i][3] and ban_co[i][0] is not O_TRONG:
            return ban_co[i][0]
        if ban_co[0][i] == ban_co[1][i] == ban_co[2][i] == ban_co[3][i] and ban_co[0][i] is not O_TRONG:
            return ban_co[0][i]

    # 2. Kiểm tra Đường chéo
    if ban_co[0][0] == ban_co[1][1] == ban_co[2][2] == ban_co[3][3] and ban_co[0][0] is not O_TRONG:
        return ban_co[0][0]
    if ban_co[0][N-1] == ban_co[1][N-2] == ban_co[2][N-3] == ban_co[3][N-4] and ban_co[0][N-1] is not O_TRONG:
        return ban_co[0][N-1]

    return None

def ket_thuc(ban_co):
    """Trả về True nếu trò chơi kết thúc (thắng hoặc hòa)."""
    if tim_nguoi_thang(ban_co) is not None:
        return True
    return not any(O_TRONG in hang for hang in ban_co)

def diem_so(ban_co):
    """Trả về điểm số: 1 nếu X thắng, -1 nếu O thắng, 0 nếu hòa."""
    nguoi_thang = tim_nguoi_thang(ban_co)
    if nguoi_thang == KY_HIEU_X:
        return 1
    elif nguoi_thang == KY_HIEU_O:
        return -1
    return 0

# --- IV. THUẬT TOÁN MINIMAX ---

def gia_tri_toi_da(trang_thai):
    """Tính giá trị tối đa (cho X) sử dụng đệ quy."""
    if ket_thuc(trang_thai):
        return diem_so(trang_thai)
    v = -math.inf
    for hanh_dong in cac_hanh_dong(trang_thai):
        v = max(v, gia_tri_toi_thieu(ket_qua_nuoc_di(trang_thai, hanh_dong)))
    return v

def gia_tri_toi_thieu(trang_thai):
    """Tính giá trị tối thiểu (cho O) sử dụng đệ quy."""
    if ket_thuc(trang_thai):
        return diem_so(trang_thai)
    v = math.inf
    for hanh_dong in cac_hanh_dong(trang_thai):
        v = min(v, gia_tri_toi_da(ket_qua_nuoc_di(trang_thai, hanh_dong)))
    return v

def minimax(ban_co):
    """Trả về nước đi tối ưu (optimal action) cho người chơi hiện tại."""
    nguoi_choi_hien_tai = nguoi_choi(ban_co)

    if nguoi_choi_hien_tai == KY_HIEU_X:
        diem_tot_nhat = -math.inf
        hanh_dong_tot_nhat = None
        for hanh_dong in cac_hanh_dong(ban_co):
            gia_tri = gia_tri_toi_thieu(ket_qua_nuoc_di(ban_co, hanh_dong))
            if gia_tri > diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
        return hanh_dong_tot_nhat

    else:
        diem_tot_nhat = math.inf
        hanh_dong_tot_nhat = None
        for hanh_dong in cac_hanh_dong(ban_co):
            gia_tri = gia_tri_toi_da(ket_qua_nuoc_di(ban_co, hanh_dong))
            if gia_tri < diem_tot_nhat:
                diem_tot_nhat = gia_tri
                hanh_dong_tot_nhat = hanh_dong
        return hanh_dong_tot_nhat

# --- V. KHỐI THỰC THI TRÒ CHƠI (MAIN) ---

if __name__ == "__main__":
    ban_co = trang_thai_ban_dau()
    print("--- TRÒ CHƠI TIC TAC TOE AI (4x4) ---")

    # 1. Chọn người chơi
    while True:
        lua_chon = input("Bạn muốn chơi với vai trò X hay O? (X/O): ").upper()
        if lua_chon in [KY_HIEU_X, KY_HIEU_O]:
            nguoi_dung = lua_chon
            ai = KY_HIEU_O if nguoi_dung == KY_HIEU_X else KY_HIEU_X
            break
        print("Lựa chọn không hợp lệ. Vui lòng nhập X hoặc O.")

    # 2. Thiết lập lượt đi đầu tiên là của người chơi
    luot_hien_tai = nguoi_dung # Gán lượt đi đầu tiên là của người dùng

    # Vòng lặp trò chơi
    while True:
        ket_thuc_game = ket_thuc(ban_co)

        # In bàn cờ hiện tại
        print("\n--- Bàn cờ hiện tại ---")
        print(np.array(ban_co))

        if ket_thuc_game:
            nguoi_thang = tim_nguoi_thang(ban_co)
            if nguoi_thang is None:
                print("Trò chơi kết thúc: HÒA.")
            else:
                print(f"Trò chơi kết thúc: {nguoi_thang} THẮNG.")
            break

        # Lượt của AI
        if luot_hien_tai == ai:
            print(f"Máy tính ({ai}) đang tính toán nước đi tối ưu...")

            # Tính nước đi
            nuoc_di_ai = minimax(ban_co)

            if nuoc_di_ai:
                # Cập nhật bàn cờ (Sử dụng hàm result chuẩn của Minimax)
                ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_ai)
                print(f"Máy tính đi: Hàng {nuoc_di_ai[0]}, Cột {nuoc_di_ai[1]}")
                luot_hien_tai = nguoi_dung # Chuyển lượt sang người dùng
            else:
                break # Hòa hoặc lỗi

        # Lượt của Người chơi
        else:
            print(f"Lượt của bạn ({nguoi_dung})")
            while True:
                try:
                    i = int(input(f"Nhập Hàng (0-{KICH_THUOC-1}): "))
                    j = int(input(f"Nhập Cột (0-{KICH_THUOC-1}): "))
                    nuoc_di_nguoi_dung = (i, j)

                    if 0 <= i < KICH_THUOC and 0 <= j < KICH_THUOC and nuoc_di_nguoi_dung in cac_hanh_dong(ban_co):
                        # Cập nhật bàn cờ
                        ban_co = ket_qua_nuoc_di(ban_co, nuoc_di_nguoi_dung)
                        luot_hien_tai = ai # Chuyển lượt sang AI
                        break
                    else:
                        print("Nước đi không hợp lệ, nằm ngoài phạm vi (0-3), hoặc ô đã bị chiếm. Vui lòng thử lại.")
                except ValueError:
                    print("Vui lòng chỉ nhập số nguyên.")

--- TRÒ CHƠI TIC TAC TOE AI (4x4) ---
Bạn muốn chơi với vai trò X hay O? (X/O): o

--- Bàn cờ hiện tại ---
[[None None None None]
 [None None None None]
 [None None None None]
 [None None None None]]
Lượt của bạn (O)
Nhập Hàng (0-3): 2
Nhập Cột (0-3): 
Vui lòng chỉ nhập số nguyên.
Nhập Hàng (0-3): 1
Nhập Cột (0-3): 2

--- Bàn cờ hiện tại ---
[[None None None None]
 [None None 'X' None]
 [None None None None]
 [None None None None]]
Máy tính (X) đang tính toán nước đi tối ưu...


KeyboardInterrupt: 